# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:

# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\ezrag\OneDrive\מסמכים\hit\שנה ב\Machine_Learning\assignment3-text_analysis\assignment3-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

We can see that the male category is the majorty in the target column.

In [11]:
df_train.groupby('gender').describe().T

gender                                                        f  \
story count                                                 178   
      unique                                                177   
      top     כזוג מאורס טרי סגרנו תאריך, אולם אירועים, ספקי...   
      freq                                                    2   

gender                                                        m  
story count                                                 575  
      unique                                                573  
      top     שבוע שעבר העליתי באופן ספונטני רעיון לנסוע עם ...  
      freq                                                    2

In [12]:
X_train = df_train['story'].copy()

In [13]:
# getting the text from each 'story' to a list.
corpus_list = X_train.to_list()
len(corpus_list)

753

## Part 1

Creating a tokenizer with the help of the hebrew tokenizer.

In [14]:
#YOUR CODE HERE
def my_tockenizer(corpus): 
    tokens = ht.tokenize(corpus)
    words = []
    for (grp, token, token_num, (start_index, end_index)) in tokens:
        if grp == 'HEBREW':
            words.append(token)
            
    return words

Mapping target categories to numerical values

In [15]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(df_train['gender'])
df_train['gender'].head(5)

y_train[:5]
y_train.shape

0    m
1    m
2    f
3    m
4    f
Name: gender, dtype: object

array([1, 1, 0, 1, 0])

(753,)

### Models Evaluation

In this section we'll train our data with various machine learning models and see which model gives us the best result.

First we'll need to find the best parmeters for our machine learning model, we figure them out by using grid search.This action is called tuning we training our model on a smaller chunck of our data  with varius parmeters and taking the best paramters, By doing so the model highly likley to preform better.

In [16]:
# this function return a GridSearch object given a pipline and various paramters options.
def tuning_param(pipe,grid_params,X,y):
    grid_search = GridSearchCV(estimator=pipe,param_grid= grid_params,scoring='f1_macro',cv=5)
    grid_search.fit(X,y)
    return grid_search
    

In [17]:
def split_data(X,y):
    #we;ll use the test data for the tuning.
    X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X,y,test_size=0.3,random_state=42)
    return X_test_bow, y_test_bow

In [18]:
#Using cross validation on the pipeline object after taking tje best params we got.
def evaluate_model(clf,X,y,scoring,cv):
    #returning the average score according to a sepecific evaluation mertric.
    score = cross_val_score(clf,X,y,scoring = scoring ,cv = cv).mean()
    return score,clf

Creating grid parameters for the models.                 

In [19]:

params_knn = {
    'vect__tokenizer':[None,my_tockenizer],
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'norm__norm':['l1', 'l2'],
    'clf__n_neighbors': list(range(3,12,2)),
} 

params_tree = {
    'vect__tokenizer':[None,my_tockenizer],
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'norm__norm':['l1', 'l2'],
    'clf__min_samples_split': list(range(2,9))
}

params_svm = {
    'vect__tokenizer':[None,my_tockenizer],
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'norm__norm':['l1', 'l2'],
    'clf__kernel':['linear', 'poly', 'rbf']
}


training a model without any parameters tunning. 

In [20]:
X,y = split_data(X_train,y_train)

In [22]:


pipe = Pipeline([
    ('vect',TfidfVectorizer()),
    ('norm',preprocessing.Normalizer()),
    ('clf', KNeighborsClassifier())
])



In [24]:
cross_val_score(pipe,X_train,y_train,scoring="f1_macro",cv=10).mean()

0.600069233712874

In [33]:
#This function run an entire process using tuning params with GridSearchCV and cross validation.
def model_process(clf_list,grid_params_list,vect_list,X_train,y_train):
    scores = []
    estimators = []
    X,y = split_data(X_train,y_train)

    for vect in vect_list: 
        for clf,grid_params in zip(clf_list,grid_params_list):
            pipe = Pipeline([
                ('vect',vect),
                ('norm',preprocessing.Normalizer()),
                ('clf', clf)
            ])
            grid_search = tuning_param(pipe,grid_params,X,y)
            #setting the pipeline with the params thast gave the best result.
            pipe.set_params(**(grid_search.best_params_))
            score,classifier = evaluate_model(pipe,X_train,y_train,'f1_macro',10)
            estimators.append(classifier)
            scores.append(score)
            
    return scores,estimators
        

In [34]:
clf_list = [KNeighborsClassifier(),tree.DecisionTreeClassifier(),svm.SVC()]
grid_params_list = [params_knn,params_tree,params_svm]
vect_list = [CountVectorizer(),TfidfVectorizer()]

In [35]:
scores,estimators = model_process(clf_list,grid_params_list,vect_list,X_train,y_train)

In [36]:
best_score_id = pd.Series(data = scores).idxmax(axis=0)
best_estimator = estimators[best_score_id]

print(f'Best estimator :', {best_estimator})
print(f'Best Score:', {max(scores)})  

Best estimator : {Pipeline(steps=[('vect', TfidfVectorizer()), ('norm', Normalizer()),
                ('clf', KNeighborsClassifier(n_neighbors=3))])}
Best Score: {0.6342123341115036}


Saving the result in a data Frame

In [40]:
vectoraizers = list()
classifiers = list()
Normalizers = list()
for estimator in estimators:
    vectoraizers.append(str(estimator.steps[0][1]))
    classifiers.append(str(estimator.steps[1][1]))
    Normalizers.append(str(estimator.steps[2][1])) 
df_results = pd.DataFrame(data = {'vect': vectoraizers,'clf':classifiers,'norm':Normalizers,'f1_average':scores})
df_results.sort_values("f1_average",ascending = False)
df_results.shape

,vect,clf,norm,f1_average
3,TfidfVectorizer(),Normalizer(),KNeighborsClassifier(n_neighbors=3),0.634
2,"CountVectorizer(ngram_range=(1, 3))",Normalizer(norm='l1'),SVC(kernel='linear'),0.619
1,"CountVectorizer(ngram_range=(1, 3))",Normalizer(),DecisionTreeClassifier(min_samples_split=3),0.597
4,TfidfVectorizer(),Normalizer(),DecisionTreeClassifier(min_samples_split=3),0.596
0,"CountVectorizer(ngram_range=(1, 3))",Normalizer(),KNeighborsClassifier(n_neighbors=3),0.545
5,TfidfVectorizer(),Normalizer(norm='l1'),SVC(kernel='linear'),0.433


(6, 4)

We can see that we got a better result then the one we got in cell 24. buit as for the other models the result are not that satisfy. Lets check the estimator on a new data.

In [81]:
best_estimator.fit(X_train,y_train)

Pipeline(steps=[('vect', TfidfVectorizer()), ('norm', Normalizer()),
                ('clf', KNeighborsClassifier(n_neighbors=3))])

In [82]:
y_predict = best_estimator.predict(df_test['story'])

In [83]:
len(y_predict)

323

In [85]:
df_predicted = pd.Series(data = y_predict,name = 'predicted_values').replace({1:'m',0:'f'})

Showing the first and last five predictions

In [97]:
df_predicted.iloc[:5]
df_predicted.iloc[-5:]


0    m
1    m
2    m
3    f
4    m
Name: predicted_values, dtype: object

318    m
319    m
320    m
321    m
322    f
Name: predicted_values, dtype: object

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [98]:
df_predicted.to_csv('classification_results.csv',index=False)